# Graphing for analysis ( video setting 1: 1/7/25)
- plotting each tree and top 3 blob sizes highlighted
- All orbits plotted aginst Ground Truth

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
from pathlib import Path
import re
from matplotlib.patches import Rectangle

# ── Configuration ──────────────────────────────────────────────────
BASE_PATH = Path(
    "/Users/sulaimanshah/Documents/Sulaiman/Orbit_Red_Blob/"
    "Data/UKE_Plot_14_DD_1_7_2025/output/Trees"
)
CSV_PATH = Path(
    "/Users/sulaimanshah/Documents/Sulaiman/Orbit_Red_Blob/"
    "Data/UKE_Plot_14_DD_1_7_2025/output/Analysis/image_log.csv"
)
GT_LOG_PATH = Path(
    "/Users/sulaimanshah/Documents/Sulaiman/Orbit_Red_Blob/"
    "Data/UKE_Plot_14_DD_1_7_2025/output/Analysis/GT_log.csv"
)
TOP_N = 3
BLOB_THRESHOLD = 200  # minimum required red blob pixels

# ── Helpers ─────────────────────────────────────────────────────────
def normalize_point(p: str) -> str:
    return str(p).strip().lower().replace(" ", "")

def extract_octant(filename: str):
    m = re.search(r'_(\d+)\.jpg', filename)
    return (int(m.group(1)) - 1) % 8 if m else None

# ── Load & clean data ──────────────────────────────────────────────
df = pd.read_csv(CSV_PATH)
gt_df = pd.read_csv(GT_LOG_PATH)

df = df.dropna(subset=["timestamp_sec", "max_blob_grey"])
df["timestamp_sec"] = pd.to_numeric(df["timestamp_sec"], errors="coerce")
df["max_blob_grey"] = pd.to_numeric(df["max_blob_grey"], errors="coerce")

df["point_norm"] = df["point"].apply(normalize_point)
gt_df["point_norm"] = gt_df["point"].apply(normalize_point)
gt_df["octant"] = gt_df["image_name"].apply(extract_octant)
gt_df = gt_df.dropna(subset=["octant"])
gt_df = gt_df[gt_df["Ripe"] > 0]

# ── Constants ──────────────────────────────────────────────────────
theta_width = 2 * np.pi / 8

# ── Iterate over each point ────────────────────────────────────────
for point_id in sorted(df["point_norm"].unique()):
    point_df = df[df["point_norm"] == point_id]
    video_name = point_df["video_name"].iloc[0]
    point_path = BASE_PATH / point_id

    top_rows = point_df.nlargest(TOP_N, "max_blob_grey")

    fig = plt.figure(figsize=(16, 20))
    gs = fig.add_gridspec(9, 6, height_ratios=[2.5] + [1]*8, width_ratios=[1.2, 1, 1, 1, 1, 0.1], hspace=0.3, wspace=0.1)

    # ─ Polar plot ─
    ax_polar = fig.add_subplot(gs[0, :5], projection="polar")
    group = point_df.sort_values("timestamp_sec")
    radii = group["max_blob_grey"].values
    angles = np.linspace(0, 2*np.pi, len(radii), endpoint=False)
    width = (2*np.pi / len(radii) if len(radii) else 0) * 0.8
    r_max = radii.max()*1.2 if len(radii) else 1

    ax_polar.bar(angles, radii, width=width, color="red", alpha=0.85)

    # Highlight top-3 with number
    top_blobs = top_rows.sort_values("max_blob_grey", ascending=False).reset_index()
    for idx, row in top_blobs.iterrows():
        angle_idx = group.index.get_loc(row["index"])
        theta = angles[angle_idx]
        r_val = row["max_blob_grey"]
        ax_polar.text(theta, r_val * 1.05, f"#{idx+1}", ha="center", va="bottom", fontsize=10, color="black")

    ripe_octants = gt_df.loc[gt_df["point_norm"] == point_id, "octant"].astype(int).unique()
    for octant in ripe_octants:
        theta_center = octant * theta_width
        ax_polar.bar(theta_center, r_max, width=theta_width, color="lightblue", alpha=0.5, zorder=0, align="center")
        if octant == 0:
            ax_polar.bar(2*np.pi, r_max, width=theta_width, color="lightblue", alpha=0.5, zorder=0, align="center")

    ax_polar.set_title(f"{point_id} | {video_name}", fontsize=12, va="bottom")
    ax_polar.set_theta_zero_location("N")
    ax_polar.set_theta_direction(1)
    ax_polar.set_rlabel_position(135)
    ax_polar.grid(True)

    # ─ GT and Predictions ─
    gt_folder = point_path / "predictions"
    gt_images = sorted(gt_folder.glob("*.jpg"))[:8]

    suffix_map = {
        0: ["30", "31", "y+", "1"],
        1: ["2", "3", "4", "5"],
        2: ["6", "7", "8", "9"],
        3: ["10", "11", "12", "13"],
        4: ["14", "15", "16", "17"],
        5: ["18", "19", "20", "21"],
        6: ["22", "23", "24", "25"],
        7: ["26", "27", "28", "29"],
    }



    # Filter top-3 predictions that pass the threshold
    top_pred_names = [f"{video_name}_zoomed_{suffix}.jpg" for suffix in sum(suffix_map.values(), [])]
    top_pred_blobs = df[
        (df["point_norm"] == point_id)
        & (df["image_name"].isin(top_pred_names))
        & (df["max_blob_grey"] >= BLOB_THRESHOLD)
    ]
    top_pred_sorted = top_pred_blobs.sort_values("max_blob_grey", ascending=False).head(3)

    # Map name → rank and blob value
    top_pred_meta = {
        row["image_name"]: (idx+1, int(row["max_blob_grey"]))
        for idx, row in top_pred_sorted.reset_index().iterrows()
    }

    for i in range(8):
        ax_gt = fig.add_subplot(gs[i+1, 0])
        if i < len(gt_images):
            img = imread(gt_images[i])
            ax_gt.imshow(img)
            ax_gt.set_title(f"GT {i+1}\n{gt_images[i].name}", fontsize=8)
        else:
            ax_gt.text(0.5, 0.5, "No GT", ha="center", va="center")
            ax_gt.set_title(f"GT {i+1}", fontsize=8)
        ax_gt.axis("off")

        pred_folder = point_path / "Orbits/images"
        for j, suffix in enumerate(suffix_map.get(i, [])):
            pred_img_name = f"{video_name}_zoomed_{suffix}.jpg"
            pred_img_path = pred_folder / pred_img_name
            ax_pred = fig.add_subplot(gs[i+1, j+1])

            # Check if it's a top prediction
            if pred_img_path.exists():
                img = imread(pred_img_path)
                ax_pred.imshow(img)

                if pred_img_name in top_pred_meta:
                    rank, blob_val = top_pred_meta[pred_img_name]
                    ax_pred.set_title(f"No{rank} | {blob_val} px\n{pred_img_name}", fontsize=8, color="red")

                    # Add visible red rectangle border
                    rect = Rectangle((0, 0), 1, 1, transform=ax_pred.transAxes,
                                    linewidth=5, edgecolor='red', facecolor='none')
                    ax_pred.add_patch(rect)
                else:
                    ax_pred.set_title(f"Pred {j+1}\n{pred_img_name}", fontsize=7)
            else:
                ax_pred.set_facecolor("whitesmoke")
                ax_pred.text(0.5, 0.5, f"{pred_img_name}\nnot found", ha="center", va="center", fontsize=6)

            ax_pred.axis("off")

    plt.tight_layout()
    plt.show()


# Graphing for analysis ( video setting 2: 1/7/25)
- plotting each tree and top 3 blob sizes highlighted
- All orbits plotted aginst Ground Truth

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
from pathlib import Path
import re
from matplotlib.patches import Rectangle

# ── Configuration ──────────────────────────────────────────────────
BASE_PATH = Path(
    "/Users/sulaimanshah/Documents/Sulaiman/Orbit_Red_Blob/"
    "Data/UKE_Plot_14_DD_1_7_2025/output/Trees"
)
CSV_PATH = Path(
    "/Users/sulaimanshah/Documents/Sulaiman/Orbit_Red_Blob/"
    "Data/UKE_Plot_14_DD_1_7_2025/output/Analysis/image_log.csv"
)
GT_LOG_PATH = Path(
    "/Users/sulaimanshah/Documents/Sulaiman/Orbit_Red_Blob/"
    "Data/UKE_Plot_14_DD_1_7_2025/output/Analysis/GT_log.csv"
)
TOP_N = 3
BLOB_THRESHOLD = 200  # minimum required red blob pixels

# ── Helpers ─────────────────────────────────────────────────────────
def normalize_point(p: str) -> str:
    return str(p).strip().lower().replace(" ", "")

def extract_octant(filename: str):
    m = re.search(r'_(\d+)\.jpg', filename)
    return (int(m.group(1)) - 1) % 8 if m else None

# ── Load & clean data ──────────────────────────────────────────────
df = pd.read_csv(CSV_PATH)
gt_df = pd.read_csv(GT_LOG_PATH)

df = df.dropna(subset=["timestamp_sec", "max_blob_grey"])
df["timestamp_sec"] = pd.to_numeric(df["timestamp_sec"], errors="coerce")
df["max_blob_grey"] = pd.to_numeric(df["max_blob_grey"], errors="coerce")

df["point_norm"] = df["point"].apply(normalize_point)
gt_df["point_norm"] = gt_df["point"].apply(normalize_point)
gt_df["octant"] = gt_df["image_name"].apply(extract_octant)
gt_df = gt_df.dropna(subset=["octant"])
gt_df = gt_df[gt_df["Ripe"] > 0]

# ── Constants ──────────────────────────────────────────────────────
theta_width = 2 * np.pi / 8

# ── Iterate over each point ────────────────────────────────────────
for point_id in sorted(df["point_norm"].unique()):
    point_df = df[df["point_norm"] == point_id]
    video_name = point_df["video_name"].iloc[0]
    point_path = BASE_PATH / point_id

    top_rows = point_df.nlargest(TOP_N, "max_blob_grey")

    fig = plt.figure(figsize=(16, 20))
    gs = fig.add_gridspec(9, 6, height_ratios=[2.5] + [1]*8, width_ratios=[1.2, 1, 1, 1, 1, 0.1], hspace=0.3, wspace=0.1)

    # ─ Polar plot ─
    ax_polar = fig.add_subplot(gs[0, :5], projection="polar")
    group = point_df.sort_values("timestamp_sec")
    radii = group["max_blob_grey"].values
    angles = np.linspace(0, 2*np.pi, len(radii), endpoint=False)
    width = (2*np.pi / len(radii) if len(radii) else 0) * 0.8
    r_max = radii.max()*1.2 if len(radii) else 1

    ax_polar.bar(angles, radii, width=width, color="red", alpha=0.85)

    # Highlight top-3 with number
    top_blobs = top_rows.sort_values("max_blob_grey", ascending=False).reset_index()
    for idx, row in top_blobs.iterrows():
        angle_idx = group.index.get_loc(row["index"])
        theta = angles[angle_idx]
        r_val = row["max_blob_grey"]
        ax_polar.text(theta, r_val * 1.05, f"#{idx+1}", ha="center", va="bottom", fontsize=10, color="black")

    ripe_octants = gt_df.loc[gt_df["point_norm"] == point_id, "octant"].astype(int).unique()
    for octant in ripe_octants:
        theta_center = octant * theta_width
        ax_polar.bar(theta_center, r_max, width=theta_width, color="lightblue", alpha=0.5, zorder=0, align="center")
        if octant == 0:
            ax_polar.bar(2*np.pi, r_max, width=theta_width, color="lightblue", alpha=0.5, zorder=0, align="center")

    ax_polar.set_title(f"{point_id} | {video_name}", fontsize=12, va="bottom")
    ax_polar.set_theta_zero_location("N")
    ax_polar.set_theta_direction(1)
    ax_polar.set_rlabel_position(135)
    ax_polar.grid(True)

    # ─ GT and Predictions ─
    gt_folder = point_path / "predictions"
    gt_images = sorted(gt_folder.glob("*.jpg"))[:8]

    suffix_map = {
        0: ["30", "31", "y+", "1"],
        1: ["2", "3", "4", "5"],
        2: ["6", "7", "8", "9"],
        3: ["10", "11", "12", "13"],
        4: ["14", "15", "16", "17"],
        5: ["18", "19", "20", "21"],
        6: ["22", "23", "24", "25"],
        7: ["26", "27", "28", "29"],
    }



    # Filter top-3 predictions that pass the threshold
    top_pred_names = [f"{video_name}_zoomed_{suffix}.jpg" for suffix in sum(suffix_map.values(), [])]
    top_pred_blobs = df[
        (df["point_norm"] == point_id)
        & (df["image_name"].isin(top_pred_names))
        & (df["max_blob_grey"] >= BLOB_THRESHOLD)
    ]
    top_pred_sorted = top_pred_blobs.sort_values("max_blob_grey", ascending=False).head(3)

    # Map name → rank and blob value
    top_pred_meta = {
        row["image_name"]: (idx+1, int(row["max_blob_grey"]))
        for idx, row in top_pred_sorted.reset_index().iterrows()
    }

    for i in range(8):
        ax_gt = fig.add_subplot(gs[i+1, 0])
        if i < len(gt_images):
            img = imread(gt_images[i])
            ax_gt.imshow(img)
            ax_gt.set_title(f"GT {i+1}\n{gt_images[i].name}", fontsize=8)
        else:
            ax_gt.text(0.5, 0.5, "No GT", ha="center", va="center")
            ax_gt.set_title(f"GT {i+1}", fontsize=8)
        ax_gt.axis("off")

        pred_folder = point_path / "Orbits/images"
        for j, suffix in enumerate(suffix_map.get(i, [])):
            pred_img_name = f"{video_name}_zoomed_{suffix}.jpg"
            pred_img_path = pred_folder / pred_img_name
            ax_pred = fig.add_subplot(gs[i+1, j+1])

            # Check if it's a top prediction
            if pred_img_path.exists():
                img = imread(pred_img_path)
                ax_pred.imshow(img)

                if pred_img_name in top_pred_meta:
                    rank, blob_val = top_pred_meta[pred_img_name]
                    ax_pred.set_title(f"No{rank} | {blob_val} px\n{pred_img_name}", fontsize=8, color="red")

                    # Add visible red rectangle border
                    rect = Rectangle((0, 0), 1, 1, transform=ax_pred.transAxes,
                                    linewidth=5, edgecolor='red', facecolor='none')
                    ax_pred.add_patch(rect)
                else:
                    ax_pred.set_title(f"Pred {j+1}\n{pred_img_name}", fontsize=7)
            else:
                ax_pred.set_facecolor("whitesmoke")
                ax_pred.text(0.5, 0.5, f"{pred_img_name}\nnot found", ha="center", va="center", fontsize=6)

            ax_pred.axis("off")

    plt.tight_layout()
    plt.show()


# Graphing for analysis ( video setting 3: 1/7/25)
- plotting each tree and top 3 blob sizes highlighted
- All orbits plotted aginst Ground Truth

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
from pathlib import Path
import re
from matplotlib.patches import Rectangle

# ── Configuration ──────────────────────────────────────────────────
BASE_PATH = Path(
    "/Users/at/Orbit_Red_Blob/"
    "Data/UKE_Plot_14_DD_16_7_2025/output/Trees"
)
CSV_PATH = Path(
    "/Users/at/Orbit_Red_Blob/"
    "Data/UKE_Plot_14_DD_16_7_2025/output/Analysis/image_log.csv"
)
GT_LOG_PATH = Path(
    "/Users/at/Orbit_Red_Blob/"
    "Data/UKE_Plot_14_DD_16_7_2025/output/Analysis/GT_log.csv"
)
TOP_N = 3
BLOB_THRESHOLD = 200  # minimum required red blob pixels

# ── Helpers ─────────────────────────────────────────────────────────
def normalize_point(p: str) -> str:
    return str(p).strip().lower().replace(" ", "")

def extract_octant(filename: str):
    m = re.search(r'_(\d+)\.jpg', filename)
    return (int(m.group(1)) - 1) % 8 if m else None

# ── Load & clean data ──────────────────────────────────────────────
df = pd.read_csv(CSV_PATH)
gt_df = pd.read_csv(GT_LOG_PATH)

df = df.dropna(subset=["timestamp_sec", "max_blob_grey"])
df["timestamp_sec"] = pd.to_numeric(df["timestamp_sec"], errors="coerce")
df["max_blob_grey"] = pd.to_numeric(df["max_blob_grey"], errors="coerce")

df["point_norm"] = df["point"].apply(normalize_point)
gt_df["point_norm"] = gt_df["point"].apply(normalize_point)
gt_df["octant"] = gt_df["image_name"].apply(extract_octant)
gt_df = gt_df.dropna(subset=["octant"])
gt_df = gt_df[gt_df["Ripe"] > 0]

# ── Constants ──────────────────────────────────────────────────────
theta_width = 2 * np.pi / 8

# ── Iterate over each point ────────────────────────────────────────
for point_id in sorted(df["point_norm"].unique()):
    point_df = df[df["point_norm"] == point_id]
    video_name = point_df["video_name"].iloc[0]
    point_path = BASE_PATH / point_id

    top_rows = point_df.nlargest(TOP_N, "max_blob_grey")

    fig = plt.figure(figsize=(16, 20))
    gs = fig.add_gridspec(9, 6, height_ratios=[2.5] + [1]*8, width_ratios=[1.2, 1, 1, 1, 1, 0.1], hspace=0.3, wspace=0.1)

    # ─ Polar plot ─
    ax_polar = fig.add_subplot(gs[0, :5], projection="polar")
    group = point_df.sort_values("timestamp_sec")
    radii = group["max_blob_grey"].values
    angles = np.linspace(0, 2*np.pi, len(radii), endpoint=False)
    width = (2*np.pi / len(radii) if len(radii) else 0) * 0.8
    r_max = radii.max()*1.2 if len(radii) else 1

    ax_polar.bar(angles, radii, width=width, color="red", alpha=0.85)

    # Highlight top-3 with number
    top_blobs = top_rows.sort_values("max_blob_grey", ascending=False).reset_index()
    for idx, row in top_blobs.iterrows():
        angle_idx = group.index.get_loc(row["index"])
        theta = angles[angle_idx]
        r_val = row["max_blob_grey"]
        ax_polar.text(theta, r_val * 1.05, f"#{idx+1}", ha="center", va="bottom", fontsize=10, color="black")

    ripe_octants = gt_df.loc[gt_df["point_norm"] == point_id, "octant"].astype(int).unique()
    for octant in ripe_octants:
        theta_center = octant * theta_width
        ax_polar.bar(theta_center, r_max, width=theta_width, color="lightblue", alpha=0.5, zorder=0, align="center")
        if octant == 0:
            ax_polar.bar(2*np.pi, r_max, width=theta_width, color="lightblue", alpha=0.5, zorder=0, align="center")

    ax_polar.set_title(f"{point_id} | {video_name}", fontsize=12, va="bottom")
    ax_polar.set_theta_zero_location("N")
    ax_polar.set_theta_direction(1)
    ax_polar.set_rlabel_position(135)
    ax_polar.grid(True)

    # ─ GT and Predictions ─
    gt_folder = point_path / "predictions"
    gt_images = sorted(gt_folder.glob("*.jpg"))[:8]

    suffix_map = {
        0: ["30", "31", "y+", "1"],
        1: ["2", "3", "4", "5"],
        2: ["6", "7", "8", "9"],
        3: ["10", "11", "12", "13"],
        4: ["14", "15", "16", "17"],
        5: ["18", "19", "20", "21"],
        6: ["22", "23", "24", "25"],
        7: ["26", "27", "28", "29"],
    }



    # Filter top-3 predictions that pass the threshold
    top_pred_names = [f"{video_name}_zoomed_{suffix}.jpg" for suffix in sum(suffix_map.values(), [])]
    top_pred_blobs = df[
        (df["point_norm"] == point_id)
        & (df["image_name"].isin(top_pred_names))
        & (df["max_blob_grey"] >= BLOB_THRESHOLD)
    ]
    top_pred_sorted = top_pred_blobs.sort_values("max_blob_grey", ascending=False).head(3)

    # Map name → rank and blob value
    top_pred_meta = {
        row["image_name"]: (idx+1, int(row["max_blob_grey"]))
        for idx, row in top_pred_sorted.reset_index().iterrows()
    }

    for i in range(8):
        ax_gt = fig.add_subplot(gs[i+1, 0])
        if i < len(gt_images):
            img = imread(gt_images[i])
            ax_gt.imshow(img)
            ax_gt.set_title(f"GT {i+1}\n{gt_images[i].name}", fontsize=8)
        else:
            ax_gt.text(0.5, 0.5, "No GT", ha="center", va="center")
            ax_gt.set_title(f"GT {i+1}", fontsize=8)
        ax_gt.axis("off")

        pred_folder = point_path / "Orbits/images"
        for j, suffix in enumerate(suffix_map.get(i, [])):
            pred_img_name = f"{video_name}_zoomed_{suffix}.jpg"
            pred_img_path = pred_folder / pred_img_name
            ax_pred = fig.add_subplot(gs[i+1, j+1])

            # Check if it's a top prediction
            if pred_img_path.exists():
                img = imread(pred_img_path)
                ax_pred.imshow(img)

                if pred_img_name in top_pred_meta:
                    rank, blob_val = top_pred_meta[pred_img_name]
                    ax_pred.set_title(f"No{rank} | {blob_val} px\n{pred_img_name}", fontsize=8, color="red")

                    # Add visible red rectangle border
                    rect = Rectangle((0, 0), 1, 1, transform=ax_pred.transAxes,
                                    linewidth=5, edgecolor='red', facecolor='none')
                    ax_pred.add_patch(rect)
                else:
                    ax_pred.set_title(f"Pred {j+1}\n{pred_img_name}", fontsize=7)
            else:
                ax_pred.set_facecolor("whitesmoke")
                ax_pred.text(0.5, 0.5, f"{pred_img_name}\nnot found", ha="center", va="center", fontsize=6)

            ax_pred.axis("off")

    plt.tight_layout()
    plt.show()



# New HSV setting 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
from pathlib import Path
import re
from matplotlib.patches import Rectangle

# ── Configuration ──────────────────────────────────────────────────
BASE_PATH = Path(
    "/Users/at/Orbit_Red_Blob/"
    "Data/UKE_Plot_14_DD_16_7_2025/output/Trees"
)
CSV_PATH = Path(
    "/Users/at/Orbit_Red_Blob/"
    "Data/UKE_Plot_14_DD_16_7_2025/output/Analysis/image_log.csv"
)
GT_LOG_PATH = Path(
    "/Users/at/Orbit_Red_Blob/"
    "Data/UKE_Plot_14_DD_16_7_2025/output/Analysis/GT_log.csv"
)
TOP_N = 3
BLOB_THRESHOLD = 4500  # minimum required red blob pixels

# ── Helpers ─────────────────────────────────────────────────────────
def normalize_point(p: str) -> str:
    return str(p).strip().lower().replace(" ", "")

def extract_octant(filename: str):
    m = re.search(r'_(\d+)\.jpg', filename)
    return (int(m.group(1)) - 1) % 8 if m else None

# ── Load & clean data ──────────────────────────────────────────────
df = pd.read_csv(CSV_PATH)
gt_df = pd.read_csv(GT_LOG_PATH)

df = df.dropna(subset=["timestamp_sec", "max_blob_grey"])
df["timestamp_sec"] = pd.to_numeric(df["timestamp_sec"], errors="coerce")
df["max_blob_grey"] = pd.to_numeric(df["max_blob_grey"], errors="coerce")

df["point_norm"] = df["point"].apply(normalize_point)
gt_df["point_norm"] = gt_df["point"].apply(normalize_point)
gt_df["octant"] = gt_df["image_name"].apply(extract_octant)
gt_df = gt_df.dropna(subset=["octant"])
gt_df = gt_df[gt_df["Ripe"] > 0]

# ── Constants ──────────────────────────────────────────────────────
theta_width = 2 * np.pi / 8

# ── Iterate over each point ────────────────────────────────────────
for point_id in sorted(df["point_norm"].unique()):
    point_df = df[df["point_norm"] == point_id]
    video_name = point_df["video_name"].iloc[0]
    point_path = BASE_PATH / point_id

    top_rows = point_df.nlargest(TOP_N, "max_blob_grey")

    fig = plt.figure(figsize=(16, 20))
    gs = fig.add_gridspec(9, 6, height_ratios=[2.5] + [1]*8, width_ratios=[1.2, 1, 1, 1, 1, 0.1], hspace=0.3, wspace=0.1)

    # ─ Polar plot ─
    ax_polar = fig.add_subplot(gs[0, :5], projection="polar")
    group = point_df.sort_values("timestamp_sec")
    radii = group["max_blob_grey"].values
    angles = np.linspace(0, 2*np.pi, len(radii), endpoint=False)
    width = (2*np.pi / len(radii) if len(radii) else 0) * 0.8
    r_max = radii.max()*1.2 if len(radii) else 1

    ax_polar.bar(angles, radii, width=width, color="red", alpha=0.85)

    # Highlight top-3 with number
    top_blobs = top_rows.sort_values("max_blob_grey", ascending=False).reset_index()
    for idx, row in top_blobs.iterrows():
        angle_idx = group.index.get_loc(row["index"])
        theta = angles[angle_idx]
        r_val = row["max_blob_grey"]
        ax_polar.text(theta, r_val * 1.05, f"#{idx+1}", ha="center", va="bottom", fontsize=10, color="black")

    ripe_octants = gt_df.loc[gt_df["point_norm"] == point_id, "octant"].astype(int).unique()
    for octant in ripe_octants:
        theta_center = octant * theta_width
        ax_polar.bar(theta_center, r_max, width=theta_width, color="lightblue", alpha=0.5, zorder=0, align="center")
        if octant == 0:
            ax_polar.bar(2*np.pi, r_max, width=theta_width, color="lightblue", alpha=0.5, zorder=0, align="center")

    ax_polar.set_title(f"{point_id} | {video_name}", fontsize=12, va="bottom")
    ax_polar.set_theta_zero_location("N")
    ax_polar.set_theta_direction(1)
    ax_polar.set_rlabel_position(135)
    ax_polar.grid(True)

    # ─ GT and Predictions ─
    gt_folder = point_path / "predictions"
    gt_images = sorted(gt_folder.glob("*.jpg"))[:8]

    suffix_map = {
        0: ["30", "31", "y+", "1"],
        1: ["2", "3", "4", "5"],
        2: ["6", "7", "8", "9"],
        3: ["10", "11", "12", "13"],
        4: ["14", "15", "16", "17"],
        5: ["18", "19", "20", "21"],
        6: ["22", "23", "24", "25"],
        7: ["26", "27", "28", "29"],
    }



    # Filter top-3 predictions that pass the threshold
    top_pred_names = [f"{video_name}_zoomed_{suffix}.jpg" for suffix in sum(suffix_map.values(), [])]
    top_pred_blobs = df[
        (df["point_norm"] == point_id)
        & (df["image_name"].isin(top_pred_names))
        & (df["max_blob_grey"] >= BLOB_THRESHOLD)
    ]
    top_pred_sorted = top_pred_blobs.sort_values("max_blob_grey", ascending=False).head(3)

    # Map name → rank and blob value
    top_pred_meta = {
        row["image_name"]: (idx+1, int(row["max_blob_grey"]))
        for idx, row in top_pred_sorted.reset_index().iterrows()
    }

    for i in range(8):
        ax_gt = fig.add_subplot(gs[i+1, 0])
        if i < len(gt_images):
            img = imread(gt_images[i])
            ax_gt.imshow(img)
            ax_gt.set_title(f"GT {i+1}\n{gt_images[i].name}", fontsize=8)
        else:
            ax_gt.text(0.5, 0.5, "No GT", ha="center", va="center")
            ax_gt.set_title(f"GT {i+1}", fontsize=8)
        ax_gt.axis("off")

        pred_folder = point_path / "Orbits/images"
        for j, suffix in enumerate(suffix_map.get(i, [])):
            pred_img_name = f"{video_name}_zoomed_{suffix}.jpg"
            pred_img_path = pred_folder / pred_img_name
            ax_pred = fig.add_subplot(gs[i+1, j+1])

            # Check if it's a top prediction
            if pred_img_path.exists():
                img = imread(pred_img_path)
                ax_pred.imshow(img)

                if pred_img_name in top_pred_meta:
                    rank, blob_val = top_pred_meta[pred_img_name]
                    ax_pred.set_title(f"No{rank} | {blob_val} px\n{pred_img_name}", fontsize=8, color="red")

                    # Add visible red rectangle border
                    rect = Rectangle((0, 0), 1, 1, transform=ax_pred.transAxes,
                                    linewidth=5, edgecolor='red', facecolor='none')
                    ax_pred.add_patch(rect)
                else:
                    ax_pred.set_title(f"Pred {j+1}\n{pred_img_name}", fontsize=7)
            else:
                ax_pred.set_facecolor("whitesmoke")
                ax_pred.text(0.5, 0.5, f"{pred_img_name}\nnot found", ha="center", va="center", fontsize=6)

            ax_pred.axis("off")

    plt.tight_layout()
    plt.show()
